# **Image Clustering Using VGG16 and CAE** 

In [ ]:
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from keras import backend
import pickle
import random
import pandas as pd
import joblib
import os
import numpy as np
from keras.models import load_model, Model
from sklearn.manifold import TSNE
from matplotlib.colors import ListedColormap
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
import imageio as io
import cv2

In [ ]:
# generate train and test data
# file_path = [file for file in os.listdir("/Users/jennysheng/Documents/museum_images/all_image_data/") if file.endswith(".jpg")]
# train_files, test_files = train_test_split(file_path, test_size = 0.05)

# train_files = pd.DataFrame(train_files,columns=['filepath'])
# test_files = pd.DataFrame(test_files,columns=['filepath'])
# train_files.to_csv('/Users/jennysheng/Documents/museum_images/train_file.csv')
# test_files.to_csv('/Users/jennysheng/Documents/museum_images/test_file.csv')
train_files = list(pd.read_csv('/Users/jennysheng/Documents/museum_images/train_file.csv')['filepath'])
test_files = list(pd.read_csv('/Users/jennysheng/Documents/museum_images/test_file.csv')['filepath'])

In [ ]:
def get_model(cae):
    if cae:
        model = load_model('/Users/jennysheng/Documents/museum_images/encoder_model.h5')
        return model
    else: 
        base_model = VGG16(weights='imagenet', include_top=True)
        model = Model(inputs=base_model.input,
                    outputs=base_model.get_layer('fc2').output)
        return model

# helper function to get all images into one dict
def get_image_files(path_to_files, size, files):
    images = []
    count = 0
    for file in files:
        if count%100 == 0:
            print("Processed " + str(count) + " files")
        image = cv2.resize(cv2.imread(path_to_files+file), size)
        images.append([file, image])
        count += 1
    return dict(images)

# helper function to get vgg image feature vectors
def vgg_feature_vector(image_array, model):
  if image_array.shape[2] == 1:
    image_array = image_array.repeat(3, axis=2)

  array_expanded = np.expand_dims(image_array, axis=0)  
  array_expanded = preprocess_input(array_expanded)
  return model.predict(array_expanded)[0,:]


# helper function to get cae image feature vectors
def cae_feature_vector(m, data, layer):
    encoded = backend.function([m.layers[0].input],[m.layers[layer].output])
    encoded_array = encoded(data[None, :,  :, :])[0]
    pooled_array = encoded_array.max(axis=-1)
    return encoded_array


# helper function to get all feature vectors into a list
def feature_vectors(images_dict, model, cae, layer):
    if cae:
        forward_vector = {}
        count = 0
        for fn, img in images_dict.items():
          if count%100 == 0:
              print("Processed " + str(count) + " vectors")
          forward_vector[fn] = cae_feature_vector(model, img, layer)
          count += 1
        return forward_vector
    else:
        forward_vector = {}
        count = 0
        for fn, img in images_dict.items():
          if count%100 == 0:
              print("Processed " + str(count) + " vectors")
          forward_vector[fn] = vgg_feature_vector(img, model)
          count += 1
        return forward_vector

In [ ]:
# initialize files, feature vectors, and models
is_cae = True # use this to toggle between the two embedding options
imgs_dict = get_image_files(path_to_files = '/Users/jennysheng/Documents/museum_images/all_image_data/', size = (224, 224), files = train_files)
model = get_model(is_cae) 

In [ ]:
img_feature_vector = feature_vectors(imgs_dict, model, is_cae, 9)

In [ ]:
if is_cae:
    image_values = np.array(list(img_feature_vector.values()))
    images = image_values.reshape(image_values.shape[0], image_values.shape[2]*image_values.shape[3]*image_values.shape[4])
else: 
    images = list(img_feature_vector.values())

In [ ]:
# f = open("/Users/jennysheng/Documents/museum_images/vgg_feature_vectors.pkl","wb")
# pickle.dump(img_feature_vector,f)
# f.close()
img_feature_vector = pd.read_pickle(r"/Users/jennysheng/Documents/museum_images/cae_feature_vectors.pkl")

In [ ]:
print(img_feature_vector.keys())

In [ ]:
# elbow plot for kmeans
fns = list(img_feature_vector.keys())
sum_of_squared_distances = []
K = range(1, 50)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(images)
    sum_of_squared_distances.append(km.inertia_)
    print("Processed cluster count K=" + str(k))

In [ ]:
# plot eblow plot
plt.plot(K, sum_of_squared_distances)
plt.title('Elbow Method for Optimal Cluster Number')
plt.xlabel('k')
plt.ylabel('Sum of Squared Distances')
plt.savefig("/Users/jennysheng/Documents/museum_images/elbow_all_museum_images_no_autoencoder.jpg")

In [ ]:
# save squared distance for reference later
with open("/Users/jennysheng/Documents/museum_images/vgg_sq_dist.txt", "wb") as fp:   #Pickling
    pickle.dump(sum_of_squared_distances, fp)

In [ ]:
# a more zoomed in look at the elbow
plt.plot(K[:10], sum_of_squared_distances[:10])
plt.title('Elbow Method for Optimal Cluster Number')
plt.xlabel('k')
plt.ylabel('Sum of Squared Distances')
plt.show()
plt.savefig("/Users/jennysheng/Documents/museum_images/elbow_all_museum_images.jpg")

In [ ]:
# run kmeans using optimal number of clusters
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, init='k-means++')
kmeans.fit(images)
y_kmeans = kmeans.predict(images)
file_names = list(imgs_dict.keys())

In [ ]:
# save kmeans clusters
np.save('/Users/jennysheng/Documents/museum_images/cluster_5_no_ae_data.npy', files)

In [ ]:
labels=kmeans.labels_
centroids = kmeans.cluster_centers_
clusters_features = []
cluster_files=[]
# iterate and collect all vectors and filenames for each cluster
for i in range(n_clusters):
    i_cluster = []
    i_labels=[]
    for idx,j in enumerate(kmeans.labels_):
        if j==i:
            i_cluster.append(images[idx])
            i_labels.append(file_names[idx])
    i_cluster = np.array(i_cluster)
    clusters_features.append(i_cluster)
    cluster_files.append(i_labels)
labels=[] # collecting all cluster ids
data=[] # collecting all feature vectors
files=[] # collecting all file names
for idx,i in enumerate(clusters_features):
    data.extend(i)
    labels.extend([idx for i in range(i.shape[0])])
    files.extend(cluster_files[idx])
print(np.array(labels).shape)
print(np.array(data).shape)
print(np.array(files).shape)

In [ ]:
image_directory_path = '/Users/jennysheng/Documents/museum_images/all_image_data/'

cluster_data = {}
for c in range(n_clusters):
    cluster_data[c] = []
for image_file, cluster in zip(file_names, y_kmeans):
    filename = image_directory_path+image_file
    cluster_data[cluster].append(filename)

In [ ]:
for c in range(0,n_clusters):
    print("cluster " + str(c) + " size: " + str(len(cluster_data[c])))

In [ ]:
# save image cluster information in dataframe
import pandas as pd
image_cluster_df = pd.DataFrame(zip(file_names, y_kmeans), columns = ["imagepath", "clusterid"])
image_cluster_df.to_csv("/Users/jennysheng/Documents/museum_images/image_cluster_df_cluster_" + str(n_clusters) + ".csv", index = False)

In [ ]:
def show_cluster(df, clusterid, num_shown, num_clusters, save, image_directory_path):
    fig = plt.figure(figsize=(14, 14))

    cluster = list(df.loc[df.clusterid == clusterid].imagepath)
    cluster_random = random.sample(cluster, num_shown)
    for i in range(num_shown):
        y = fig.add_subplot(6, 5, i+1)
        img = mpimg.imread(image_directory_path + cluster[i])
        y.imshow(img)
        plt.title('cluster ' + str(clusterid))
        y.axes.get_xaxis().set_visible(False)
        y.axes.get_yaxis().set_visible(False)
    if save:
        plt.savefig("/Users/jennysheng/Documents/museum_images/"+ str(num_clusters) + "_clusters_total_cluster_" + str(clusterid) + ".jpg")

In [ ]:
show_cluster(image_cluster_df, 0, 30, n_clusters, True, image_directory_path)

In [ ]:
show_cluster(image_cluster_df, 1, 30, n_clusters, True, image_directory_path)

In [ ]:
show_cluster(image_cluster_df, 2, 30, n_clusters, True, image_directory_path)

In [ ]:
show_cluster(image_cluster_df, 3, 30, n_clusters, True, image_directory_path)

In [ ]:
show_cluster(image_cluster_df, 4, 30, n_clusters, True, image_directory_path)

In [ ]:
show_cluster(image_cluster_df, 5, 30, n_clusters, True, image_directory_path)

In [ ]:
show_cluster(image_cluster_df, 6, 30, n_clusters, True)

In [ ]:
# save kmeans model
kmeans_file = '/Users/jennysheng/Documents/museum_images/kmeans_model_cluster_' + str(n_clusters) + '_1.pkl'
joblib.dump(kmeans,kmeans_file)

# **Image Cluster Metadata Analysis**

In [ ]:
# data analysis of each cluster
merged = pd.read_csv("/Users/jennysheng/Documents/museum_images/merged_final_1.csv")
clusters = pd.read_csv("/Users/jennysheng/Documents/museum_images/image_cluster_df_cluster_" + str(n_clusters) + ".csv")

In [ ]:
# helper function to get cluster meta data
def get_cluster_meta_data(merged_df, cluster_df, clusterid):
    cluster = list(cluster_df.loc[cluster_df.clusterid == clusterid].imagepath)
    all_repositories = {"met_":"Metropolitan Museum of Art, New York, NY", "mia_":"Minneapolis Institute of Art", "puam":"Princeton University Art Museum", "moma":"The Museum of Modern Art", "cmoa":"Carnegie Museum of Art"}
    data = pd.DataFrame()
    for i in range(len(cluster)):
        imagepath = cluster[i]
        id = imagepath[:-9]
        repo = imagepath[-8:-4]
        meta_data = merged_df.loc[merged_df.repository == all_repositories[repo]]
        meta_data_with_id = meta_data.loc[meta_data.objectid == id]
        if len(meta_data_with_id) == 0:
            meta_data_with_id = meta_data.loc[meta_data.objectid == int(id)]
        data = data.append(meta_data_with_id)
    print(len(data))
    return data

In [ ]:
cluster_data = []
for i in range(n_clusters):
    print("Cluster " + str(i) + " data:")
    cluster_data_i = get_cluster_meta_data(merged, clusters, i)
    cluster_data.append(cluster_data_i)
    print(cluster_data_i.repository.value_counts())

In [ ]:
# helper function to get value counts for characteristics or fields
def get_characteristic_frequency_in_cluster(cluster_data, characteristic):
    if characteristic == "tags":
        met = cluster_data.loc[cluster_data.repository == "Metropolitan Museum of Art, New York, NY"]
        met_characteristics = pd.DataFrame(met[characteristic])
        met_characteristics["objectid"] = met_characteristics.index
        met_characteristics = pd.melt(met_characteristics, id_vars=['objectid'], value_vars=met_characteristics.columns.values[:-1])
        multihot = pd.get_dummies(met_characteristics.set_index('objectid')['value']).max(level=0).reset_index()
    if characteristic == "accessionyear":
        cluster_data_characteristics = pd.DataFrame(cluster_data[characteristic])
        cluster_data_characteristics["objectid"] = cluster_data_characteristics.index
        cluster_data_characteristics = pd.melt(cluster_data_characteristics, id_vars=['objectid'], value_vars=cluster_data_characteristics.columns.values[:-1])
        multihot = pd.get_dummies(cluster_data_characteristics.set_index('objectid')['value']).max(level=0).reset_index()
    else:
        cluster_data_characteristics = cluster_data[characteristic].str.split("|", expand=True)
        cluster_data_characteristics["objectid"] = cluster_data_characteristics.index
        cluster_data_characteristics = pd.melt(cluster_data_characteristics, id_vars=['objectid'], value_vars=cluster_data_characteristics.columns.values[:-1])
        multihot = pd.get_dummies(cluster_data_characteristics.set_index('objectid')['value']).max(level=0).reset_index()
    return multihot

def cluster_characteristics(clusterid, merged_df, cluster_df, characteristic, num_shown, single_cluster_data):
    cluster_data = single_cluster_data[clusterid]
    multihot_cluster = get_characteristic_frequency_in_cluster(cluster_data, characteristic)
    counts = dict(multihot_cluster[[c for c in multihot_cluster.columns if not c == "objectid"]].sum())
    counts = sorted(counts.items(), key=lambda kv: kv[1], reverse=True)
    print(counts[:num_shown])

In [ ]:
for i in range(n_clusters):
    print("Cluster " + str(i))
    cluster_characteristics(i, merged, clusters, "culture", 20, cluster_data)

In [ ]:
for i in range(n_clusters):
    print("Cluster " + str(i))
    cluster_characteristics(i, merged, clusters, "tags", 20, cluster_data)

In [ ]:
for i in range(n_clusters):
    print("Cluster " + str(i))
    cluster_characteristics(i, merged, clusters, "artistdisplayname", 20, cluster_data)

In [ ]:
for i in range(n_clusters):
    print("Cluster " + str(i))
    cluster_characteristics(i, merged, clusters, "classification", 20, cluster_data)

In [ ]:
for i in range(n_clusters):
    print("Cluster " + str(i))
    cluster_characteristics(i, merged, clusters, "accessionyear", 20, cluster_data)

# **t-SNE Analysis**

In [ ]:
# perform dimensionality reduction to 2D using t-SNE
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, verbose=2).fit_transform(images)
print("Done!")

In [ ]:
import pandas as pd

tx = tsne[:, 0]
tx = (tx - np.min(tx))/(np.max(tx) - np.min(tx))
ty = tsne[:, 1]
ty = (ty - np.min(ty))/(np.max(ty) - np.min(ty))

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111)

colors =["blue", "red", "yellow", "green", "purple", "orange", "pink", "magenta", "teal", "brown"]
image_cluster_df = pd.read_csv("/Users/jennysheng/Documents/museum_images/image_cluster_df_cluster_" + str(n_clusters) + ".csv")


for clusterid in range(n_clusters):
    cluster = image_cluster_df.loc[image_cluster_df.clusterid == clusterid]
    cluster_dict = dict(zip(file_names, y_kmeans))
    indices = [i for i, (k, v) in enumerate(cluster_dict.items()) if v == clusterid]
    ax.scatter(tx[indices], ty[indices], c=colors[clusterid], label=clusterid)

ax.legend()
ax.set_title("2D T-SNE Visualization of Clusters")

plt.savefig("/Users/jennysheng/Documents/museum_images/cae_cluster_visualization_cluster_" + str(n_clusters) + ".jpg")

# **Find Similar Images Using K-NN**

In [ ]:
# find optimal number of neighbors
from sklearn.model_selection import GridSearchCV
grid_params = {'n_neighbors': range(0, 20)}
X = tsne  
y = y_kmeans
knn = KNeighborsClassifier()
gs = GridSearchCV(knn, grid_params, verbose = 1, cv = 3, n_jobs = 3)
gs.fit(X,y)
print (gs.best_score_)
print (gs.best_params_)
print (gs.best_estimator_)

In [ ]:
# run knn model on optimal number of neighbors and save knn model
n_neighbors=12
knn = KNeighborsClassifier(n_neighbors=n_neighbors,algorithm='ball_tree',n_jobs=-1)
knn.fit(np.array(data),np.array(labels))
knn_file = '/Users/jennysheng/Documents/museum_images/knn_model_clusters_' + str(n_clusters) + "_neighbors_" + str(n_neighbors) + "_1.pkl"
joblib.dump(knn,knn_file)

# **Testing**

In [ ]:
# find similar neigbors
def predictions(query, N, model, knn, files, image_dir_path, size):
    image = cv2.resize(cv2.imread(image_dir_path+query), size)
    feature = feature_vector(image, model)
    res = knn.kneighbors(feature.reshape(1,-1),return_distance=True,n_neighbors=N)
    return [files[i] for i in list(res[1][0])[1:][:N+1]]

In [ ]:
# show similar images in a grid-like plot
def show_similar_images(images):
    fig = plt.figure(figsize=(14, 14))
    for i in range(len(images)):
        y = fig.add_subplot(6, 5, i+1)
        img = mpimg.imread("/Users/jennysheng/Documents/museum_images/all_image_data/" + images[i])
        y.imshow(img)
        y.axes.get_xaxis().set_visible(False)
        y.axes.get_yaxis().set_visible(False)

In [ ]:
num = 10 # feel free to change this number to any index between 0 and 499 to see results
img = mpimg.imread("/Users/jennysheng/Documents/museum_images/all_image_data/" + test_files[num])
plt.imshow(img)

In [ ]:
similar_images = predictions(test_files[num], 7, model, knn, files, image_directory_path, (224, 224))
show_similar_images(similar_images)